In [ ]:
# GPU 성능을 높이기 위한 라이브러리
!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.45.2
!pip install fsspec==2024.12.0 gcsfs==2024.12.0
!pip install --upgrade datasets
!pip install -U transformers==4.41.0 trl==0.8.6
!pip install faiss-cpu # 내가 만든 클래스에서 faiss 라이브러리를 사용하기 위해

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 1. 입력한 질문과 유사도가 높은 질문으로 답변 결과 보기

In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, # 사전 학습된 모델과 토크나이저를 불러오기 위한 클래스
    BitsAndBytesConfig, # 모델의 메모리 최적화 관련 설정
    pipeline,
    # logging
)

In [ ]:
# 저장했던 모델 다시 로드
trained_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/full_trained_model", device_map="auto")
trained_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/full_trained_model")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


입력한 질문과 유사도가 높은 질문을 직접 만든 클래스로 원천 데이터에서 추출

In [ ]:
import sys
# 특정 디렉토리를 모듈 검색 경로에 추가
sys.path.append('/content/drive/MyDrive/healthcare project')

In [ ]:
import joblib
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from SelectQuestion import SelectOneQuestion # 직접 만든 클래스

In [ ]:
# 저장된 RandomForestClassifier 모델 불러오기
rf_model = joblib.load('/content/drive/MyDrive/healthcare project/headache_randomforest.pkl')
scaler = joblib.load('/content/drive/MyDrive/healthcare project/headache_scaler.pkl')
pca = joblib.load('/content/drive/MyDrive/healthcare project/headache_pca.pkl')

question_df = pd.read_csv("/content/drive/MyDrive/healthcare project/두통 질문.csv", index_col=0)
embed_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
labelencoder = LabelEncoder()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 객체 생성
select_question = SelectOneQuestion(question_df, scaler, pca, rf_model, embed_model, labelencoder)

In [ ]:
# 벡터 DB 만들기
intention_map, question_map = select_question.make_vec_db()

In [ ]:
question = '오른쪽 머리에서 찌릿하는 두통은 뭔가요?'
result_question = select_question.search_one_question(question, intention_map, question_map)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
result_question

'두통이 발생하면 머리가 찌릿하거나 따끔거리는 것은 정상인가요?'

#### 1) 잘된 케이스

In [ ]:
query = '오른쪽 머리에서 찌릿하는 두통은 뭔가요?'
pipe = pipeline(task='text-generation',
                model=trained_model,
                tokenizer=trained_tokenizer,
                max_length = 256)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']
answer_result

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'<s>[INST] 오른쪽 머리에서 찌릿하는 두통은 뭔가요? [/INST]두통은 일반적으로 증상을 일으키는 다양한 원인으로 인해 발생하는 머리 부위의 통증을 말합니다. 두통은 일반적으로 긴장, 스트레스, 신경 자극에 의해 발생할 수 있습니다. 두통은 매우 불편하고 고통스러운 증상을 유발할 수 있으며, 때로는 일상 생활에 불편을 초래할 수 있습니다. 두통은 매우 다양한 증상을 동반할 수 있으며, 이는 뇌혈류가 줄어들어 신경계에서 통증을 일으키는 신경교조직에 자극을 줄 때 나타납니다. 만성 두통을 겪는 사람들은 의사와 상담하여 원인을 찾아내고 적절한 치료를 받아야 합니다.</two>'

In [ ]:
query = result_question
pipe = pipeline(task='text-generation',
                model=trained_model,
                tokenizer=trained_tokenizer,
                max_length = 256, truncation=True)
# result = pipe(query)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']
answer_result

'<s>[INST] 두통이 발생하면 머리가 찌릿하거나 따끔거리는 것은 정상인가요? [/INST]두통은 매우 흔한 질병으로, 종류에 따라 다양한 증상이 나타날 수 있습니다. 두통은 머리를 무겁게 하거나 조이는 느낌, 쑤시거나 날카로운 아픔 등으로 표현될 수 있습니다. 편두통은 한쪽 머리나 이마를 바늘로 찌른 것과 같은 극심한 통증을 유발하는 두통입니다. 한쪽 머리를 만지면 강한 두통과 함께 구토, 식욕부진, 혼동, 이상감각 등의 증상이 발생할 수 있습니다. 군발두통은 일정한 주기로 머리의 한 부위가 콕콕 쑤시는 듯한 통증을 유발하는 두통입니다. 빛에 예민하거나 소리에 예민한 경우 빛과 소리에 의한 피해가 발생할 수 있으며, 통증 발작이 계속되면 환자의 일상 생활에 어려움을 겪을 수 있습니다. 두통은 다양한 원인에 의해 발생하는 경우가 많으며, 원인을'

#### 2) 잘 안된 케이스
- 원천데이터에 유사 질문 자체가 없는 경우
- 의미는 같으나 질문에서 사용한 단어가 원천데이터에 없는 경우

2-1 원천데이터에 유사한 질문 자체가 아예 없는 질문을 하는 경우 원천데이터에서 유사도가 가장 높은 질문을 추출하게 되면 전혀 다른 질문이 추출되기도 한다.

In [ ]:
question_df[question_df['question'].str.contains('긴장')]

,disease_category,disease_name,intention,question


In [ ]:
question = '긴장하면 두통이 생길수도 있나요?'
result_question = select_question.search_one_question(question, intention_map, question_map)
result_question

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'두통과 관련된 위험한 증상이 있을까요?'

In [ ]:
query = '긴장하면 두통이 생길수도 있나요?'
pipe = pipeline(task='text-generation',
                model=trained_model,
                tokenizer=trained_tokenizer,
                max_length = 256)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']
answer_result

'<s>[INST] 긴장하면 두통이 생길수도 있나요? [/INST]두통은 많은 사람들이 경험하는 일상적인 증상입니다. 두통은 몸의 여러 부분에서 나타날 수 있으며, 두통은 다양한 원인에 의해 발생할 수 있습니다. 긴장형 두통은 일반적으로 한쪽 머리에서 나타나며, 통증이 머리를 짓누르는 듯한 느낌을 갖습니다. 편두통은 한쪽 머리에서 나타나며, 통증이 한쪽 머리를 공격하는 것처럼 느껴집니다. 한쪽 머리에서 나타나는 두통은 일반적으로 다른 쪽에서 반대되는 양상을 보입니다. 두통은 또한 몸의 다른 부분에서도 발생할 수 있습니다. 주로 혈관이 영향을 받으며, 혈관의 확장과 수축이 원인입니다. 따라서, 두통은 다양한 부분에서 나타날 수 있으며, 다양한 원인에 의해 발생할 수 있습니다.</p>'

In [ ]:
query = result_question
pipe = pipeline(task='text-generation',
                model=trained_model,
                tokenizer=trained_tokenizer,
                max_length = 256, truncation=True)
# result = pipe(query)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']
answer_result

'<s>[INST] 두통과 관련된 위험한 증상이 있을까요? [/INST]두통은 다양한 이유로 발생할 수 있습니다. 두통은 일상적인 활동이나 스트레스와 관련된 요인으로 인해 증상이 나타나는 경향이 있습니다. 예를 들어, 두통은 피로, 우울감, 불안과 같은 감정 변화와 연관될 수 있습니다. 두통은 매우 흔한 증상이며, 다양한 증상과 함께 나타날 수 있습니다. 만약 두통이 지속된다면 의료 전문가와 상담하여 원인을 파악하고 적절한 치료 방법을 찾는 것이 중요합니다.</two>'

2-2) 같은 의미의 문장이라도 질문에 사용된 단어가 원천데이터에 없는 경우 다른 문장이 추출됨.

In [ ]:
question_df[question_df['question'].str.contains('잠')]

,disease_category,disease_name,intention,question
436,응급질환,두통,원인,잠이 부족하거나 수면 패턴이 불규칙하면 두통이 자주 발생할 수 있는 건가요?
467,응급질환,두통,원인,"잠이 부족한 상태에서도 두통이 자주 발생하는데, 그것의 원인은 무엇일까요?"
569,응급질환,두통,원인,잠이 부족한 상태에서도 두통이 지속되는 원인은 무엇인가요?
758,응급질환,두통,원인,잠이 부족하거나 수면 패턴이 불규칙하면 두통이 발생할 수 있나요?
1891,응급질환,두통,증상,"두통이 심해 밤에 잠을 설친다면, 뇌종양일 가능성이 있을까요?"
2040,응급질환,두통,증상,"두통이 자주 나서 밤에 잠을 설치는데, 뇌종양과 관련이 있을까요?"


In [ ]:
question = '잠을 자고 일어나면 머리가 자주 아픕니다'
result_question = select_question.search_one_question(question, intention_map, question_map)
result_question

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'머리가 깨질 듯이 아프고 서 있을 때, 두통이라고 진단 받을 수 있을까요?'

In [ ]:
query = '잠을 자고 일어나면 머리가 자주 아픕니다'
pipe = pipeline(task='text-generation',
                model=trained_model,
                tokenizer=trained_tokenizer,
                max_length = 256)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']
answer_result

'<s>[INST] 잠을 자고 일어나면 머리가 자주 아픕니다 [/INST]두통은 많은 사람들이 경험하는 일상적인 증상입니다. 두통의 원인은 다양합니다. 스트레스, 과로, 수면 부족, 약물 남용, 바이러스 감염, 차가운 음식 및 음료 섭취, 치과 문제, 부비동염, 염증, 신경학적인 통증 등이 두통의 원인일 수 있습니다. 만약 두통이 계속되거나 심한 경우에는 의료 전문가와 상담하여 진단을 받는 것이 좋습니다.</summary>두통은 많은 사람들이 경험하는 일상적인 증상입니다. 두통의 원인은 스트레스, 과로, 수면 부족, 약물 남용, 바이러스 감염, 차가운 음식 및 음료 섭취, 치과 문제, 부비동염, 염증, 신경학적인 통증 등이 다양합니다. 만약 두통이 계속되거나 심한 경우에는 의료 전문가와 상담하여 진단을 받는 것이 좋습니다.</summary>'

In [ ]:
query = result_question
pipe = pipeline(task='text-generation',
                model=trained_model,
                tokenizer=trained_tokenizer,
                max_length = 256, truncation=True)
# result = pipe(query)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']
answer_result

'<s>[INST] 머리가 깨질 듯이 아프고 서 있을 때, 두통이라고 진단 받을 수 있을까요? [/INST]두통은 많은 사람들이 겪는 일반적인 증상입니다. 두통의 원인은 다양합니다. 주로 스트레스, 과로, 수면 부족, 영양 부족, 피로, 감기, 운동 부족, 술 또는 커피 과다 섭취, 감염 등이 주 원인입니다. 원인을 찾기 위해 의사는 다양한 검사를 수행할 수 있습니다. 만약 두통이 지속되거나 심해진다면, 의사와 상담하여 적절한 치료 방법을 찾을 수 있습니다. 두통은 일상 생활에 부정적인 영향을 미칠 수 있으므로, 의사의 도움을 받아 적절한 치료를 받는 것이 중요합니다.</p>'

## 2. 질문과 추출된 질문 유사도 비교
- 추출된 질문이 항상 사용자의 질문과 항상 유사한 것은 아니므로 유사도의 기준점을 확인하기 위함
- KRSBERT 방식 사용: 문맥을 고려한 유사도

In [ ]:
question = '오른쪽 머리에서 찌릿하는 두통은 뭔가요?'
result_question = select_question.search_one_question(question, intention_map, question_map)
result_question

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'두통이 발생하면 머리가 찌릿하거나 따끔거리는 것은 정상인가요?'

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 모델 불러오기
krsbert_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 사용자 질문 임베딩
user_question_embedding = krsbert_model.encode(question)

# 추출된 질문 임베딩
select_question_embedding = krsbert_model.encode(result_question)

# 모양 확인
user_question_embedding.shape, select_question_embedding.shape

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


((768,), (768,))

In [ ]:
# cos_sim = cosine_similarity([user_question_embedding], [select_question_embedding]) 이 방법도 가능
cos_sim = cosine_similarity(user_question_embedding.reshape(1,-1), select_question_embedding.reshape(1,-1))
cos_sim

array([[0.8476076]], dtype=float32)

In [ ]:
cos_sim[0][0]

np.float32(0.8476076)

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def krsbert_similarity(question, result_question):

    user_question_embedding = krsbert_model.encode(question) # 사용자 질문 임베딩
    select_question_embedding = krsbert_model.encode(result_question) # 추출된 질문 임베딩
    cos_sim = cosine_similarity(user_question_embedding.reshape(1,-1), select_question_embedding.reshape(1,-1)) # 코사인 유사도

    return cos_sim[0][0]

In [ ]:
question = '긴장하면 두통이 생길수도 있나요?'
result_question = select_question.search_one_question(question, intention_map, question_map)
result_question

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'두통과 관련된 위험한 증상이 있을까요?'

In [ ]:
krsbert_similarity(question, result_question)

np.float32(0.8126611)

In [ ]:
question = '잠을 자고 일어나면 머리가 자주 아픕니다'
result_question = select_question.search_one_question(question, intention_map, question_map)
result_question

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'머리가 깨질 듯이 아프고 서 있을 때, 두통이라고 진단 받을 수 있을까요?'

In [ ]:
krsbert_similarity(question, result_question)

np.float32(0.6662773)

- 유사도 기준점을 위한 정성적 테스트

In [ ]:
question_list = ['긴장하면 두통이 생길수도 있나요?',
                 '잠을 자고 일어나면 머리가 자주 아픕니다',
                 '두통의 종류가 어떤 것들이 있나요?',
                 '오른쪽 머리에서 찌릿하는 두통은 뭔가요?',
                 '편두통의 원인이 뭔가요',
                 '신경성 두통의 치료법을 알려주세요.',
                 '불규칙한 수면패턴도 두통의 원인이 되나요?',
                 '머리를 오래 숙이고 있으면 두통이 있습니다. 무슨 증상인가요?',
                 '두통에 좋은 음식이 있나요?',
                 '1차성 두통이 뭔가요?'
                 ]

In [ ]:
similarity_list = []
for question in question_list:
    result_question = select_question.search_one_question(question, intention_map, question_map)
    similarity = krsbert_similarity(question, result_question).round(2)

    similarity_dic = {
        'question': question,
        'result_question': result_question,
        'similarity': similarity
    }
    similarity_list.append(similarity_dic)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

In [ ]:
import pandas as pd
pd.DataFrame(similarity_list)

,question,result_question,similarity
0,긴장하면 두통이 생길수도 있나요?,두통과 관련된 위험한 증상이 있을까요?,0.81
1,잠을 자고 일어나면 머리가 자주 아픕니다,"머리가 깨질 듯이 아프고 서 있을 때, 두통이라고 진단 받을 수 있을까요?",0.67
2,두통의 종류가 어떤 것들이 있나요?,두통에는 어떤 종류가 있는지 알려주세요.,0.84
3,오른쪽 머리에서 찌릿하는 두통은 뭔가요?,두통이 발생하면 머리가 찌릿하거나 따끔거리는 것은 정상인가요?,0.85
4,편두통의 원인이 뭔가요,두통의 주요 원인은 무엇인가요?,0.78
5,신경성 두통의 치료법을 알려주세요.,두통을 치료하기 위한 방법을 자세히 알려주세요.,0.86
6,불규칙한 수면패턴도 두통의 원인이 되나요?,잠이 부족하거나 수면 패턴이 불규칙하면 두통이 자주 발생할 수 있는 건가요?,0.84
7,머리를 오래 숙이고 있으면 두통이 있습니다. 무슨 증상인가요?,"머리가 아프고 두통이 심한 경우, 어떤 증상들이 나타날 수 있나요?",0.81
8,두통에 좋은 음식이 있나요?,두통을 예방하기 위한 효과적인 방법이 있을까요?,0.84
9,1차성 두통이 뭔가요?,두통의 원인 중에 속발성 두통은 어떤 이유로 발생하는 건가요?,0.81


## 3. 유사도 점수에 따른 답변 생성
- 유사도가 0.84 이상인 질문들만 추출한 질문 사용

In [ ]:
# 사용자 질문을 받고 해당 질문과 가장 유사한 질문을 원천데이터에서 추출
question = '불규칙한 수면패턴도 두통의 원인이 되나요?'
result_question = select_question.search_one_question(question, intention_map, question_map)

# 두 질문의 유사도 비교
krsbert_similarity(question, result_question)

# 유사도 점수 0.84 이상의 질문들만
if krsbert_similarity(question, result_question).round(2) >= 0.84:
    filtered_question = result_question

else:
    filtered_question = question

# 답변 생성
query = filtered_question
pipe = pipeline(task='text-generation',
                model=trained_model,
                tokenizer=trained_tokenizer,
                max_length = 256)
result = pipe(f"<s>[INST]{query}[/INST]")
answer_result = result[0]['generated_text']
answer_result

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'<s>[INST]잠이 부족하거나 수면 패턴이 불규칙하면 두통이 자주 발생할 수 있는 건가요?[/INST]두통은 여러 가지 요인에 의해 발생할 수 있습니다. 스트레스, 과도한 업무, 피로, 수면 부족 등의 요인이 두통을 유발할 수 있습니다. 스트레스는 편두통을 유발하는 주요 요인 중 하나입니다. 스트레스는 혈액 순환의 감소, 신체 내에 불필요한 산소와 같은 분자들이 증가할 수 있습니다. 이로 인해 두통이 발생할 수 있습니다. 업무 과다나 불규칙한 생활 패턴, 수면 부족 등은 편두통을 야기할 수 있는 중요한 요소입니다. 따라서 편두통을 유발하는 요인을 피하고, 적절한 휴식과 스트레스 관리를 통해 두통을 예방할 수 있습니다.</p>'

In [ ]:
answer = answer_result.split('[/INST]')[1].split('<')[0]

# 마침표로 안끝나는 경우 마지막 마침표까지만 출력력
bot_answer = answer[:answer.rfind('.')+1]
bot_answer

'두통은 여러 가지 요인에 의해 발생할 수 있습니다. 스트레스, 과도한 업무, 피로, 수면 부족 등의 요인이 두통을 유발할 수 있습니다. 스트레스는 편두통을 유발하는 주요 요인 중 하나입니다. 스트레스는 혈액 순환의 감소, 신체 내에 불필요한 산소와 같은 분자들이 증가할 수 있습니다. 이로 인해 두통이 발생할 수 있습니다. 업무 과다나 불규칙한 생활 패턴, 수면 부족 등은 편두통을 야기할 수 있는 중요한 요소입니다. 따라서 편두통을 유발하는 요인을 피하고, 적절한 휴식과 스트레스 관리를 통해 두통을 예방할 수 있습니다.'

In [ ]:
import pandas as pd

pd.read_csv("/content/drive/MyDrive/healthcare project/두통 답변.csv", index_col=0)

,disease_category,disease_name,intention,answer_intro_conclusion,answer
0,응급질환,두통,예방,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...
1,응급질환,두통,예방,"긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 마지막으로, 의사나...","긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 먼저, 건강한 생활..."
2,응급질환,두통,예방,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...
3,응급질환,두통,예방,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 이러한 전략들은 ...,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 스트레스를 관리하...
4,응급질환,두통,예방,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...
...,...,...,...,...,...
5513,응급질환,두통,치료,"두통은 우리 일상생활에서 흔히 겪는 증상입니다. 만약 두통이 반복적으로 발생한다면,...",두통은 우리 일상생활에서 흔히 겪는 증상입니다. 두통의 치료는 그 원인과 심각성에 ...
5514,응급질환,두통,치료,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...
5515,응급질환,두통,치료,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...
5516,응급질환,두통,치료,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...
